In [1]:
import logging
import can
from time import sleep

# Initialize logging
logging.basicConfig(
    level=logging.ERROR, format="%(asctime)s - %(levelname)s - %(message)s"
)

# Initialize the CAN bus
bus = can.interface.Bus(interface="pcan", channel="PCAN_USBBUS1", bitrate=1000000)
########################################
# Import the CAN bus libraries for the gripper and Xiaomi micro motor
########################################
from gripper_control import GripperControl
from pcan_cybergear import CANMotorController

In [2]:
# The CAN ID of the gripper driver is 1
gripper = GripperControl(bus, addr=1)

# Modify the maximum output voltage in closed-loop mode to adjust the maximum clamping force. The unit is mV, and the maximum value can be 5000mV.
gripper.set_max_output_voltage(5000)

# Automatically initialize the gripper's open and close positions
gripper.init_position()

# Close the gripper
# gripper.close_gripper()

# sleep(3)

# Automatically adjust the position to clear any jamming
# gripper.clear_clog()

# Open the gripper
# gripper.open_gripper()

In [3]:
gripper.set_max_output_voltage(3000)

In [4]:
gripper.close_gripper()

In [5]:
gripper.open_gripper()

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from cv2 import getTickCount, getTickFrequency
import ollama
import re
import time
import json  # Import the json module
from concurrent.futures import ThreadPoolExecutor
from collections import defaultdict

# Load the YOLOv11 model
model = YOLO("weights/yolo11s.pt")

# Capture video from the camera
cap = cv2.VideoCapture(0)

# Global variables
force_cache = {}  # Cache for force values
last_query_time = {}  # Record the last query time for each object
QUERY_INTERVAL = 5  # Query interval for each object (seconds)
current_force = None  # Current force value in use
executor = ThreadPoolExecutor(max_workers=1)  # For asynchronous force value queries

# Define the list of interested class IDs
INTERESTED_CLASSES = {
    39: 'bottle',     # LayerD
    41: 'cup',        # LayerD
    44: 'spoon',      # LayerD
    47: 'apple',      # LayerC
    49: 'orange',     # LayerB
    50: 'broccoli',   # LayerC
    51: 'carrot',     # LayerC
    65: 'remote',     # LayerD
    67: 'cell phone', # LayerD
    73: 'book',       # LayerD
    76: 'scissors',   # LayerD
    79: 'toothbrush', # LayerD
}

def get_force_value(object_name):
    message_content = f"""
    Based on the following dictionaries, determine which level the object '[object_name]' is most similar to, and output only the corresponding value of that level as a single number. If the object is not listed, use your best judgment to determine the closest match and output the value for that level. Do not include any code or text, only a number.

    Dictionary of hierarchical structures and corresponding object arrays:

    "LayerA": ["cup"],  # Soft Texture Items
    "LayerB": ["orange"],  # Moderately Soft Texture Items
    "LayerC": ["broccoli", "carrot", "apple"],  # Hard Texture Items
    "LayerD": ["bottle", "remote", "cell phone", "book"],  # Hard items
    "LayerE": ["spoon", "scissors", "toothbrush"]  # Very Hard Items

    Dictionary of hierarchical structures and corresponding values:

        'LayerA': 1000,
        'LayerB': 2000,
        'LayerC': 3000,
        'LayerD': 4000,
        'LayerE': 5000


    Based on the typical weight characteristics of '{object_name}', determine which level it belongs to and output only the corresponding value.
    """

    try:
        response = ollama.chat(model="llama3.2", stream=False, messages=[{
            "role": "assistant",
            "content": message_content
        }], options={"temperature": 0.5})
        content = response['message']['content']
        numbers = re.findall(r'\d+', content)
        return int(numbers[0]) if numbers else 5000
    except Exception as e:
        print(f"Force value query error: {e}")
        return 5000  # Return default value when error occurs

def update_force_value(object_name):
    """Wrapper function for asynchronous force value updates"""
    force_value = get_force_value(object_name)
    force_cache[object_name] = force_value
    last_query_time[object_name] = time.time()
    return force_value

def get_current_force(detected_objects):
    """Get the current force value to be used"""
    global current_force
    
    if not detected_objects:
        current_force = None
        return None

    current_time = time.time()
    for obj_name in detected_objects:
        # If the object is not in the cache
        if obj_name not in force_cache:
            # Synchronously update the force value (only on the first detection)
            force_value = get_force_value(object_name)
            force_cache[obj_name] = force_value
            last_query_time[obj_name] = time.time()
        # If the object is in the cache but the query interval has passed
        elif current_time - last_query_time.get(obj_name, 0) > QUERY_INTERVAL:
            # Asynchronously update the force value
            executor.submit(update_force_value, obj_name)    
    # Use the maximum force value from the cache
    max_force = max([force_cache.get(obj, 5000) for obj in detected_objects])
    current_force = max_force
    return current_force

def write_to_file(detected_objects, force):
    """Append detected object names and force to a text file in JSON array format"""
    # Read existing data from the file (if it exists)
    try:
        with open("detected_objects.json", "r") as file:
            data = json.load(file)  # Load existing JSON data
    except (FileNotFoundError, json.JSONDecodeError):
        # Initialize an empty list if the file does not exist or contains invalid JSON
        data = []
    
    # Append new detection results to the data
    for obj in detected_objects:
        data.append({"object": obj, "force": force})  # Store each object and force as a dictionary
    
    # Write the updated data back to the file in JSON format
    with open("detected_objects.json", "w") as file:
        json.dump(data, file, indent=4)  # Use indent=4 for readability

# Record the start time of the program
start_time = time.time()

while cap.isOpened():
    loop_start = getTickCount()
    success, frame = cap.read()
    
    if success:
        # Add class filtering
        results = model.predict(source=frame, classes=list(INTERESTED_CLASSES.keys()))
        result = results[0]
        boxes = result.boxes
        
        # Get all detected objects in the current frame (only interested classes)
        detected_objects = []
        for cls_id in boxes.cls:
            cls_id = int(cls_id)
            if cls_id in INTERESTED_CLASSES:
                detected_objects.append(INTERESTED_CLASSES[cls_id])
        
        # Update force value
        force = get_current_force(detected_objects)
        
        # Write detected object names and force to a text file in JSON format
        if detected_objects and force:
            write_to_file(detected_objects, force)
        
        # Display detected objects and force value on the frame
        annotated_frame = results[0].plot()
        
        # Calculate FPS
        loop_time = getTickCount() - loop_start
        fps = int(getTickFrequency() / loop_time)
        
        # Display information
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(annotated_frame, f"FPS: {fps}", (10, 30), font, 1, (0, 0, 255), 2)
        if force:
            cv2.putText(annotated_frame, f"Force: {force}", (10, 70), font, 1, (0, 255, 0), 2)
        
        # Display detected object names
        y_offset = 110
        for obj in detected_objects:
            cv2.putText(annotated_frame, f"Detected: {obj}", (10, y_offset), font, 0.7, (255, 0, 0), 2)
            y_offset += 30
        
        cv2.imshow('Object Detection', annotated_frame)
    
    # Check if more than 20 seconds have passed
    # if time.time() - start_time > 20:
    #     break

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

executor.shutdown(wait=False)
cap.release()
cv2.destroyAllWindows()


0: 480x640 (no detections), 175.6ms
Speed: 1.8ms preprocess, 175.6ms inference, 54.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 13.4ms
Speed: 2.8ms preprocess, 13.4ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 16.5ms
Speed: 0.0ms preprocess, 16.5ms inference, 2.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 14.8ms
Speed: 0.0ms preprocess, 14.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 18.9ms
Speed: 0.0ms preprocess, 18.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 14.8ms
Speed: 1.4ms preprocess, 14.8ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 24.1ms
Speed: 2.3ms preprocess, 24.1ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 20.5ms
Speed: 2.2ms preprocess, 20.5m

In [2]:
import json

def print_latest_force():
    """Print the force value from the latest entry in the detected_objects.json file."""
    try:
        # Read the JSON file
        with open("detected_objects.json", "r") as file:
            data = json.load(file)  # Load JSON data
        
        # Check if the data is empty
        if not data:
            print("No data found in detected_objects.json.")
            return
        
        # Get the latest entry
        latest_entry = data[-1]  # The last element in the list is the latest entry
        
        # Extract the force value
        force = latest_entry.get("force")
        
        # Print the force value
        if force is not None:
            print(f"Latest force value: {force}")
        else:
            print("No force value found in the latest entry.")
    
    except FileNotFoundError:
        print("File detected_objects.json not found.")
    except json.JSONDecodeError:
        print("Invalid JSON format in detected_objects.json.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Call the function to print the latest force value
print_latest_force()

Latest force value: 4000


In [4]:
print(force)

None
